<a href="https://colab.research.google.com/github/kuppamcharan0606/Business-Analyst-Time-Series-Modeling-/blob/master/Business_Analyst_(Time_Series_Modeling).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [193]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from lazypredict.Supervised import LazyRegressor

In [194]:
df = pd.read_excel('https://docs.google.com/spreadsheets/d/1BSO9_52GLGwe2WrtB3TpS9QqXGk3mM0L/export?format=xlsx')

In [ ]:
df.head()

In [ ]:
df.reset_index(inplace=True)
df.set_index('Date', inplace=True)
df.drop(['Australia _export','Canada_export', 'Saudi_export', 'Usa_export', 'India_export',
       'Russia_export', 'South_Africa_export', 'Turkey', 'Brazil',
       'France_export', 'Germeny_export', 'United Kingdome_export',
       'China_export', 'Australia _import', 'Canada_import', 'Saudi_import',
       'Usa_import', 'India_import', 'Russia_import', 'South_Africa_import',
       'Turkey_import', 'Brazil_import', 'France_import', 'Germeny_import',
       'United Kingdome_import', 'China_import', 'Japan_import',
       'South_korea_import'], axis=1, inplace=True)
print(df.columns)

In [196]:
train_data = df.iloc[:-12]
test_data = df.iloc[-12:]

In [197]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Assuming you have a target column named 'target_column'
target_column = 'Domestic Market (Contract) Blow Molding, Low'

# Split the data into train and test sets
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]

# Fit and evaluate ARIMA model
def arima_model(train, test):
    history = train.values
    predictions = []
    for t in range(len(test)):
        model = ARIMA(history, order=(2,1,1))  # Set appropriate values for p, d, q
        model_fit = model.fit()
        forecast = model_fit.forecast()[0]
        predictions.append(forecast)
        history = np.append(history, test[t])
    mse = mean_squared_error(test, predictions)
    rmse = np.sqrt(mse)
    return predictions, rmse

# Train and evaluate the ARIMA model
arima_predictions, arima_rmse = arima_model(train_data[target_column], test_data[target_column])


In [198]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Assuming you have a target column named 'target_column'
target_column = 'Domestic Market (Contract) Blow Molding, Low'

# Split the data into train and test sets
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]

# Scale the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train_data = scaler.fit_transform(train_data[[target_column]])
scaled_test_data = scaler.transform(test_data[[target_column]])

# Define the lookback window size
lookback = 10

# Function to create input features and target variable for LSTM
def create_dataset(data, lookback):
    X, y = [], []
    for i in range(len(data)-lookback):
        X.append(data[i:i+lookback, 0])
        y.append(data[i+lookback, 0])
    return np.array(X), np.array(y)

# Create the input features and target variable for LSTM
X_train, y_train = create_dataset(scaled_train_data, lookback)
X_test, y_test = create_dataset(scaled_test_data, lookback)

# Reshape the input features for LSTM (samples, time steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(lookback, 1)))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the LSTM model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# Make predictions using the trained model
train_predictions = model.predict(X_train)
lstm_predictions = model.predict(X_test)

# Inverse scale the predictions
train_predictions = scaler.inverse_transform(train_predictions)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

Epoch 1/100
7/7 [==============================] - 1s 112ms/step - loss: 0.2042
Epoch 2/100
7/7 [==============================] - 1s 145ms/step - loss: 0.0350
Epoch 3/100
7/7 [==============================] - 1s 152ms/step - loss: 0.0329
Epoch 4/100
7/7 [==============================] - 1s 152ms/step - loss: 0.0222
Epoch 5/100
7/7 [==============================] - 1s 111ms/step - loss: 0.0202
Epoch 6/100
7/7 [==============================] - 1s 115ms/step - loss: 0.0190
Epoch 7/100
7/7 [==============================] - 1s 118ms/step - loss: 0.0169
Epoch 8/100
7/7 [==============================] - 1s 114ms/step - loss: 0.0170
Epoch 9/100
7/7 [==============================] - 1s 112ms/step - loss: 0.0164
Epoch 10/100
7/7 [==============================] - 1s 115ms/step - loss: 0.0158
Epoch 11/100
7/7 [==============================] - 1s 113ms/step - loss: 0.0159
Epoch 12/100
7/7 [==============================] - 1s 113ms/step - loss: 0.0153
Epoch 13/100
7/7 [===================

In [199]:
# Fit the model
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
X_train, X_test, y_train, y_test = train_test_split(train_data.drop('Domestic Market (Contract) Blow Molding, Low', axis=1), train_data['Domestic Market (Contract) Blow Molding, Low'], test_size=0.2, random_state=42)
lazy_predictions = reg.fit(X_train, X_test, y_train, y_test)

# Select only the predictions for the test data
lazy_predictions_df = pd.DataFrame({'Prediction': lazy_predictions})
lazy_predictions = lazy_predictions_df.iloc[-12:]['Prediction'].values

100%|██████████| 42/42 [00:02<00:00, 15.59it/s]


In [200]:
# Calculate evaluation metrics
def directional_accuracy(y_true, y_pred):
    return np.mean(np.sign(y_true.diff().iloc[1:]) == np.sign(y_pred.diff().iloc[1:]))

print('ARIMA Metrics:')
print('RMSE:', mean_squared_error(test_data['Domestic Market (Contract) Blow Molding, Low'], arima_predictions, squared=False))
print('MSE:', mean_squared_error(test_data['Domestic Market (Contract) Blow Molding, Low'], arima_predictions))
print('R2:', r2_score(test_data['Domestic Market (Contract) Blow Molding, Low'], arima_predictions))
print('')

print('LSTM Metrics:')
y_true = test_data['Domestic Market (Contract) Blow Molding, Low'].values[-46:]
print('RMSE:', mean_squared_error(y_true, lstm_predictions, squared=False))
print('MSE:', mean_squared_error(y_true, lstm_predictions))
print('R2:', r2_score(y_true, lstm_predictions))
print('')


ARIMA Metrics:
RMSE: 5.224083911017431
MSE: 27.291052709351174
R2: 0.9086019789741463

LSTM Metrics:
RMSE: 6.984124514626741
MSE: 48.7779952358102
R2: 0.8652085806423722



In [201]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# Split the data into train and test sets
train_size = int(len(df) * 0.7)
train_data, test_data = df.iloc[:train_size], df.iloc[train_size:]

# Prepare the train and test data
X_train, y_train = train_data.drop(columns=['Domestic Market (Contract) Blow Molding, Low']), train_data['Domestic Market (Contract) Blow Molding, Low']
X_test, y_test = test_data.drop(columns=['Domestic Market (Contract) Blow Molding, Low']), test_data['Domestic Market (Contract) Blow Molding, Low']

# Build the XGBoost regressor model
model = XGBRegressor()
model.fit(X_train, y_train)

# Make predictions using the XGBoost regressor model
xgb_predictions = model.predict(X_test)

# Calculate RMSE, MSE, and R2 for the model
rmse_xgb = np.sqrt(mean_squared_error(y_test, xgb_predictions))
mse_xgb = mean_squared_error(y_test, xgb_predictions)
r2_xgb = r2_score(y_test, xgb_predictions)

In [202]:
print('XGBOOST Metrics:')
print('RMSE:', rmse_xgb)
print('MSE:', mse_xgb)
print('R2:', r2_xgb)
print('')

XGBOOST Metrics:
RMSE: 8.968185105599208
MSE: 80.42834408829148
R2: 0.6515841853132337

